<a href="https://colab.research.google.com/github/otakunoichin/test/blob/main/AI%E7%B6%B2%E8%86%9CVerify%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step1: 環境準備
import os, re, glob, math, time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as TF

print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))

In [ ]:
# Step2: Google Drive をマウント
from google.colab import drive
drive.mount('/content/drive')

# ★あなたのフォルダに合わせる
DATA_ROOT = "/content/drive/MyDrive/ai_dataset"
IMG_DIR   = os.path.join(DATA_ROOT, "images")
LBL_DIR   = os.path.join(DATA_ROOT, "labels")

print("IMG_DIR:", IMG_DIR)
print("LBL_DIR:", LBL_DIR)
print("images sample:", os.listdir(IMG_DIR)[:5])
print("labels sample:", os.listdir(LBL_DIR)[:5])

In [ ]:
# ファイル名から数字IDを抜き出す関数（例: "001.jpg"→1）
def extract_id(path):
    base = os.path.basename(path)
    m = re.match(r"(\d+)\.(jpg|jpeg|png|tif|tiff)$", base, flags=re.IGNORECASE)
    if not m:
        return None
    return int(m.group(1))  # 001 -> 1

# 画像側（images）は jpg/tif混在なので両方拾う
img_paths = glob.glob(os.path.join(IMG_DIR, "*.jpg")) + \
            glob.glob(os.path.join(IMG_DIR, "*.jpeg")) + \
            glob.glob(os.path.join(IMG_DIR, "*.png")) + \
            glob.glob(os.path.join(IMG_DIR, "*.tif")) + \
            glob.glob(os.path.join(IMG_DIR, "*.tiff"))

lbl_paths = glob.glob(os.path.join(LBL_DIR, "*.tif")) + \
            glob.glob(os.path.join(LBL_DIR, "*.tiff")) + \
            glob.glob(os.path.join(LBL_DIR, "*.png"))  # 念のため

img_map = {}
for p in img_paths:
    i = extract_id(p)
    if i is None:
        continue
    # 同じIDが複数拡張子で存在したら jpg優先などにしたい場合はここで制御
    if i not in img_map:
        img_map[i] = p

lbl_map = {}
for p in lbl_paths:
    i = extract_id(p)
    if i is None:
        continue
    if i not in lbl_map:
        lbl_map[i] = p

common_ids = sorted(set(img_map.keys()) & set(lbl_map.keys()))
only_img   = sorted(set(img_map.keys()) - set(lbl_map.keys()))
only_lbl   = sorted(set(lbl_map.keys()) - set(img_map.keys()))

print("images files:", len(img_paths), "unique IDs:", len(img_map))
print("labels files:", len(lbl_paths), "unique IDs:", len(lbl_map))
print("paired IDs:", len(common_ids))
print("only in images:", len(only_img), "example:", only_img[:10])
print("only in labels:", len(only_lbl), "example:", only_lbl[:10])

# ペア一覧（最初の10個）
for i in common_ids[:10]:
    print(i, "->", os.path.basename(img_map[i]), os.path.basename(lbl_map[i]))

In [ ]:
def load_grayscale(path):
    # 画像はグレースケール前提（FAF）
    img = Image.open(path)
    img = img.convert("L")
    arr = np.array(img)
    return arr

def show_pair(idx=0):
    img_id = common_ids[idx]
    img = load_grayscale(img_map[img_id])
    msk = load_grayscale(lbl_map[img_id])

    print("ID:", img_id)
    print("image:", img.dtype, img.shape, "min/max:", img.min(), img.max(), "mean:", float(img.mean()))
    print("mask :", msk.dtype, msk.shape, "min/max:", msk.min(), msk.max(), "unique(sample):", np.unique(msk)[:10])

    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(img, cmap="gray"); ax[0].set_title("FAF image"); ax[0].axis("off")
    ax[1].imshow(msk, cmap="gray"); ax[1].set_title("Mask (raw)"); ax[1].axis("off")
    ax[2].imshow(img, cmap="gray")
    ax[2].imshow(msk, cmap="jet", alpha=0.35)
    ax[2].set_title("Overlay"); ax[2].axis("off")
    plt.show()

show_pair(0)

In [ ]:
PATCH_ROOT = os.path.join(DATA_ROOT, "patches")
PATCH_IMG_DIR = os.path.join(PATCH_ROOT, "images")
PATCH_LBL_DIR = os.path.join(PATCH_ROOT, "labels")
os.makedirs(PATCH_IMG_DIR, exist_ok=True)
os.makedirs(PATCH_LBL_DIR, exist_ok=True)

TARGET_SIZE = 4096
PATCH_SIZE  = 512

def reflect_pad_to(arr, target=4096):
    h, w = arr.shape
    assert h <= target and w <= target
    pad_h = target - h
    pad_w = target - w
    # 上下左右に均等に（奇数なら下/右を多め）
    top = pad_h // 2
    bottom = pad_h - top
    left = pad_w // 2
    right = pad_w - left
    arr_pad = np.pad(arr, ((top, bottom), (left, right)), mode="reflect")
    return arr_pad, (top, bottom, left, right)

def save_patch(arr, path):
    Image.fromarray(arr).save(path)

# 実行（全ペアをパッチ化）
total = 0
for img_id in common_ids:
    img = load_grayscale(img_map[img_id])
    msk = load_grayscale(lbl_map[img_id])

    img_pad, pad_info = reflect_pad_to(img, TARGET_SIZE)
    msk_pad, _        = reflect_pad_to(msk, TARGET_SIZE)

    # 0/255 マスクを 0/1 に寄せたい場合（学習安定）
    # ※ Step3でunique見てから必要ならON
    # msk_pad = (msk_pad > 0).astype(np.uint8) * 255

    for r in range(TARGET_SIZE // PATCH_SIZE):
        for c in range(TARGET_SIZE // PATCH_SIZE):
            y0 = r * PATCH_SIZE
            x0 = c * PATCH_SIZE
            img_p = img_pad[y0:y0+PATCH_SIZE, x0:x0+PATCH_SIZE]
            msk_p = msk_pad[y0:y0+PATCH_SIZE, x0:x0+PATCH_SIZE]

            out_img = os.path.join(PATCH_IMG_DIR, f"image_{img_id:03d}_row{r}_col{c}.png")
            out_msk = os.path.join(PATCH_LBL_DIR, f"mask_{img_id:03d}_row{r}_col{c}.png")
            save_patch(img_p, out_img)
            save_patch(msk_p, out_msk)
            total += 1

print("total patches:", total)
print("expected per image:", (TARGET_SIZE//PATCH_SIZE)*(TARGET_SIZE//PATCH_SIZE))

In [ ]:
# 確認：パッチ数
patch_imgs = sorted(glob.glob(os.path.join(PATCH_IMG_DIR, "*.png")))
patch_msks = sorted(glob.glob(os.path.join(PATCH_LBL_DIR, "*.png")))
print("patch image count:", len(patch_imgs))
print("patch mask  count:", len(patch_msks))

# 確認：1枚見てサイズ
sample_img = np.array(Image.open(patch_imgs[0]).convert("L"))
sample_msk = np.array(Image.open(patch_msks[0]).convert("L"))
print("patch img shape:", sample_img.shape, "dtype:", sample_img.dtype, "min/max:", sample_img.min(), sample_img.max())
print("patch msk shape:", sample_msk.shape, "dtype:", sample_msk.dtype, "unique(sample):", np.unique(sample_msk)[:10])

# 確認：パディングが自然か（端に近いパッチ vs 中央パッチ）
def show_patch(name_contains):
    p = [x for x in patch_imgs if name_contains in os.path.basename(x)][0]
    mid = p.replace("/images/", "/labels/").replace("image_", "mask_")
    img = np.array(Image.open(p).convert("L"))
    msk = np.array(Image.open(mid).convert("L"))
    fig, ax = plt.subplots(1,3, figsize=(15,5))
    ax[0].imshow(img, cmap="gray"); ax[0].set_title(os.path.basename(p)); ax[0].axis("off")
    ax[1].imshow(msk, cmap="gray"); ax[1].set_title("mask"); ax[1].axis("off")
    ax[2].imshow(img, cmap="gray"); ax[2].imshow(msk, cmap="jet", alpha=0.35); ax[2].set_title("overlay"); ax[2].axis("off")
    plt.show()

# 例：row0_col0（端）と row3_col3（中央）など
show_patch("row0_col0")
show_patch("row3_col3")